In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

2.7.0


#### 1. Load the dataset

In [2]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [4]:
train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  

for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [15]:
type(train_data)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

#### 2. Preprocessing

In [5]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

In [7]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [8]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


#### 3.  Model Creation

In [17]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 120, 16)           160000    
                                                                 
 flatten_2 (Flatten)         (None, 1920)              0         
                                                                 
 dense_4 (Dense)             (None, 6)                 11526     
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [18]:
num_epochs = 10
model1.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 2s 2ms/step - loss: 0.4984 - accuracy: 0.7400 - val_loss: 0.3530 - val_accuracy: 0.8440
Epoch 2/10
782/782 [==============================] - 2s 2ms/step - loss: 0.2497 - accuracy: 0.9029 - val_loss: 0.3559 - val_accuracy: 0.8429
Epoch 3/10
782/782 [==============================] - 1s 2ms/step - loss: 0.1096 - accuracy: 0.9690 - val_loss: 0.4344 - val_accuracy: 0.8312
Epoch 4/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0306 - accuracy: 0.9954 - val_loss: 0.5144 - val_accuracy: 0.8280
Epoch 5/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0091 - accuracy: 0.9991 - val_loss: 0.5788 - val_accuracy: 0.8287
Epoch 6/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0033 - accuracy: 0.9998 - val_loss: 0.6412 - val_accuracy: 0.8288
Epoch 7/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.6870 - val_accuracy: 0.8290
Epoch 

In [19]:
e = model1.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


#### 3.1 Model with GlobalAveragePooling1D instead of flatten

In [21]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 120, 16)           160000    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 flatten_4 (Flatten)         (None, 16)                0         
                                                                 
 dense_8 (Dense)             (None, 6)                 102       
                                                                 
 dense_9 (Dense)             (None, 1)                 7         
                                                                 
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
________________________________________________

In [22]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 2s 2ms/step - loss: 0.5569 - accuracy: 0.7535 - val_loss: 0.3921 - val_accuracy: 0.8423
Epoch 2/10
782/782 [==============================] - 2s 2ms/step - loss: 0.3373 - accuracy: 0.8618 - val_loss: 0.3323 - val_accuracy: 0.8585
Epoch 3/10
782/782 [==============================] - 1s 2ms/step - loss: 0.2722 - accuracy: 0.8894 - val_loss: 0.3330 - val_accuracy: 0.8568
Epoch 4/10
782/782 [==============================] - 2s 2ms/step - loss: 0.2354 - accuracy: 0.9079 - val_loss: 0.3418 - val_accuracy: 0.8555
Epoch 5/10
782/782 [==============================] - 1s 2ms/step - loss: 0.2092 - accuracy: 0.9216 - val_loss: 0.3618 - val_accuracy: 0.8502
Epoch 6/10
782/782 [==============================] - 1s 2ms/step - loss: 0.1874 - accuracy: 0.9323 - val_loss: 0.3923 - val_accuracy: 0.8424
Epoch 7/10
782/782 [==============================] - 2s 2ms/step - loss: 0.1698 - accuracy: 0.9386 - val_loss: 0.4179 - val_accuracy: 0.8382
Epoch 

In [23]:
e1 = model1.layers[0]
weights1 = e1.get_weights()[0]
print(weights1.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


#### 4.Saving the result to Tensorflow Projector compatible datatype

In [24]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [25]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [26]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
